In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/17"
pmc = "13004032" # 店舗ID
SPREADSHEET_KEY = '1ErvaAeN76qBeTCajdI7IEffcNmc7OxGtL1jEWAjRMtQ' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 601 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+123@gmail.com"
user_pw = "paruparu21"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p2row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value
s2row = worksheet.cell(11,  30).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p2table = []
s20table = []
s5table = []
s2table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "111":
                    p1table.append(li)
                if unit == "22":
                    p2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "111":
                        p1table.append(li)
                    if unit == "22":
                        p2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "555":
                    s5table.append(li)
                if unit == "222":
                    s2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "555":
                        s5table.append(li)
                    if unit == "222":
                        s2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.2RAW'!F" + str(int(p2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p2table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S2RAW'!F" + str(int(s2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s2table})

['08/17', '025553', '1', '52051.0', '21367.0', '58', '8']
['08/17', '025553', '2', '46923.0', '-21064.0', '12', '5']
['08/17', '025553', '3', '48718.0', '19862.0', '55', '11']
['08/17', '025553', '4', '46667.0', '10533.0', '41', '11']
['08/17', '025553', '5', '46154.0', '2107.0', '32', '12']
['08/17', '025877', '6', '39231.0', '-2106.0', '38', '18']
['08/17', '025877', '7', '43077.0', '11436.0', '60', '22']
['08/17', '025877', '8', '43590.0', '16552.0', '64', '18']
['08/17', '025877', '9', '39744.0', '-3911.0', '36', '14']
['08/17', '025877', '10', '21026.0', '20765.0', '55', '10']
['08/17', '025877', '11', '43077.0', '-2707.0', '41', '17']
['08/17', '025877', '12', '47179.0', '12038.0', '67', '20']
['08/17', '025877', '13', '40000.0', '-10231.0', '25', '16']
['08/17', '025521', '14', '34103.0', '6019.0', '42', '16']
['08/17', '025521', '15', '36923.0', '3311.0', '41', '15']
['08/17', '025521', '16', '43846.0', '10834.0', '59', '22']
['08/17', '025521', '17', '36154.0', '-902.0', '35',

['08/17', '025308', '142', '8205.0', '-2406.0', '5', '3']
['08/17', '025540', '143', '2564.0', '603.0', '7', '1']
['08/17', '025661', '144', '769.0', '-902.0', '0', '0']
['08/17', '025764', '145', '14872.0', '-7823.0', '3', '1']
['08/17', '025421', '146', '1538.0', '-1203.0', '0', '0']
['08/17', '025507', '147', '15641.0', '-3610.0', '19', '8']
['08/17', '025739', '148', '1282.0', '-902.0', '1', '1']
['08/17', '025308', '149', '12051.0', '-7823.0', '3', '3']
['08/17', '025741', '150', '23590.0', '-7823.0', '16', '8']
['08/17', '025741', '151', '15385.0', '1506.0', '19', '5']
['08/17', '025741', '152', '23590.0', '-6319.0', '16', '9']
['08/17', '025741', '153', '14615.0', '4515.0', '22', '10']
['08/17', '025741', '154', '25128.0', '-6619.0', '16', '7']
['08/17', '025573', '155', '29487.0', '5418.0', '25', '5']
['08/17', '025593', '156', '0', '0', '2', '1']
['08/17', '025286', '157', '7692.0', '-3911.0', '3', '1']
['08/17', '025789', '158', '3333.0', '-3008.0', '0', '0']
['08/17', '02578

['08/17', '025846', '284', '0', '0', '4', '2']
['08/17', '025844', '285', '513.0', '-902.0', '0', '0']
['08/17', '025844', '286', '0', '0', '0', '0']
['08/17', '025895', '287', '25385.0', '21367.0', '31', '5']
['08/17', '025895', '288', '19744.0', '-4212.0', '11', '3']
['08/17', '025895', '289', '30256.0', '9631.0', '28', '8']
['08/17', '025895', '290', '24359.0', '-3008.0', '12', '1']
['08/17', '025895', '291', '30769.0', '-18356.0', '7', '3']
['08/17', '025895', '292', '29231.0', '-20763.0', '2', '2']
['08/17', '025895', '293', '33590.0', '15348.0', '33', '8']
['08/17', '025895', '294', '38205.0', '-7823.0', '19', '4']
['08/17', '025825', '295', '0', '0', '0', '0']
['08/17', '025825', '296', '2308.0', '-2106.0', '0', '0']
['08/17', '025825', '297', '5385.0', '-4814.0', '0', '0']
['08/17', '025825', '298', '5128.0', '2107.0', '5', '3']
['08/17', '025825', '299', '3590.0', '2107.0', '4', '2']
['08/17', '025825', '300', '4103.0', '-3610.0', '0', '0']
['08/17', '025825', '301', '6667.0',

['08/17', '025670', '433', '0', '0', '0', '0']
['08/17', '025670', '434', '0', '0', '0', '0']
['08/17', '025670', '435', '0', '0', '0', '0']
['08/17', '025670', '436', '0', '0', '0', '0']
['08/17', '025670', '437', '513.0', '-1203.0', '0', '0']
['08/17', '025781', '438', '4103.0', '6621.0', '9', '1']
['08/17', '025854', '439', '0', '0', '0', '0']
['08/17', '025854', '440', '0', '0', '0', '0']
['08/17', '025579', '441', '9744.0', '3010.0', '6', '2']
['08/17', '025579', '442', '15385.0', '-4814.0', '4', '1']
['08/17', '025579', '443', '10256.0', '-6018.0', '1', '1']
['08/17', '025579', '444', '1795.0', '603.0', '1', '1']
['08/17', '025579', '445', '6154.0', '-5115.0', '0', '0']
['08/17', '025579', '446', '8462.0', '-4513.0', '1', '1']
['08/17', '025579', '447', '5641.0', '-4212.0', '0', '0']
['08/17', '025579', '448', '8718.0', '-4814.0', '1', '1']
['08/17', '025579', '449', '7949.0', '-6319.0', '0', '0']
['08/17', '025579', '450', '3846.0', '-3008.0', '0', '0']
['08/17', '025579', '451'

['08/17', '120093', '648', '2300', '2222.0', '23', '0', '23']
['08/17', '120093', '649', '1738', '-1111.0', '7', '0', '7']
['08/17', '120093', '650', '490', '509.0', '7', '0', '7']
['08/17', '120108', '651', '1709', '-463.0', '0', '0', '8']
['08/17', '120108', '652', '2464', '1620.0', '0', '0', '24']
['08/17', '120108', '653', '350', '-370.0', '0', '0', '--']
['08/17', '120073', '654', '1090', '-139.0', '0', '4', '7']
['08/17', '120073', '655', '5481', '-648.0', '0', '16', '36']
['08/17', '120073', '656', '1624', '741.0', '0', '4', '12']
['08/17', '120073', '657', '1239', '-556.0', '0', '5', '7']
['08/17', '120073', '658', '2749', '-2037.0', '0', '8', '13']
['08/17', '120073', '659', '4904', '1620.0', '0', '17', '38']
['08/17', '120073', '660', '1694', '-370.0', '0', '5', '11']
['08/17', '120127', '661', '55', '0', '0', '0', '--']
['08/17', '120127', '662', '414', '-370.0', '0', '0', '1']
['08/17', '120127', '663', '50', '0', '0', '0', '--']
['08/17', '120127', '664', '277', '-93.0', '

['08/17', '120025', '785', '2710', '1111.0', '14', '6', '--']
['08/17', '120025', '786', '156', '-93.0', '0', '0', '--']
['08/17', '120025', '787', '255', '-185.0', '0', '0', '--']
['08/17', '120025', '788', '194', '-231.0', '0', '0', '--']
['08/17', '120025', '789', '2780', '-648.0', '9', '2', '--']
['08/17', '120025', '790', '150', '-185.0', '0', '0', '--']
['08/17', '120025', '791', '197', '139.0', '1', '0', '--']
['08/17', '120025', '792', '173', '-185.0', '0', '0', '--']
['08/17', '120025', '793', '794', '-278.0', '1', '3', '--']
['08/17', '120025', '794', '2914', '-556.0', '7', '7', '--']
['08/17', '120025', '795', '3180', '1157.0', '16', '8', '--']
['08/17', '120025', '796', '108', '-93.0', '0', '0', '--']
['08/17', '120025', '797', '4643', '370.0', '20', '8', '--']
['08/17', '120025', '798', '679', '-185.0', '3', '0', '--']
['08/17', '120025', '799', '343', '-231.0', '0', '1', '--']
['08/17', '120025', '800', '1707', '-602.0', '5', '1', '--']
['08/17', '120021', '801', '109', '

{'spreadsheetId': '1ErvaAeN76qBeTCajdI7IEffcNmc7OxGtL1jEWAjRMtQ',
 'updates': {'spreadsheetId': '1ErvaAeN76qBeTCajdI7IEffcNmc7OxGtL1jEWAjRMtQ',
  'updatedRange': 'S2RAW!F2115:M2128',
  'updatedRows': 14,
  'updatedColumns': 8,
  'updatedCells': 112}}